# 网络搭建
### 四种方法：
- 层层堆叠：利用torch.nn库中的各类层函数完成网络的构建，是最基本的构建方法。
- Sequential: 该操作可以将各个层封装到Sequential构建的容器当中，更利于呈现复杂的网络结构。
- Sequential+add_module(): 首先构建空的Sequential容器，然后利用add_module()函数一一添加相应的层。
- Sequential+OrderedDict：对层结构命名的简单方法
### 步骤：
（1）class类的命名：要继承nn.Module的特征  
（2）init部分：完成层的构建<u>（以上四种方法只有在该部分有区别）</u>  
（3）forward部分：完成输入在神经网络中前向传播的过程  

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import OrderedDict

#方法一：层层堆叠:利用torch.nn逐一构建层结构
class Net1(nn.Module):
    def __init__(self) -> None:
        super(Net1,self).__init__()
        self.conv1 = nn.Conv2d(3,32,3,1,1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.dense1 = nn.Linear(32 * 3 * 3, 128)
        self.dense2 = nn.Linear(128,10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0),-1)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        # x = F.max_pool2d(F.relu(self.conv(x)),2)
        # x = x.view(x.size(0),-1)
        # x = F.relu(self.dense1(x))
        # x = self.dense2(x)
        return x
net1 = Net1()
print(net1)

Net1(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dense1): Linear(in_features=288, out_features=128, bias=True)
  (dense2): Linear(in_features=128, out_features=10, bias=True)
)


#### Note:  
(1) 如果用torch.functional来构建层结构，在print时不会呈现出对应的结构。  
(2) 如果采用层层堆叠的方式构建，print的结构只会按照实际命名的先后顺序出现，不代表实际的x的前向传播顺序。  
(3) 利用Sequential构建的网络结构，基本上代表了前向传播的顺序（维度操作是不会出现在网络结构中的）

In [4]:
# 方法二：Sequential时序构建法，但是各层无名字，只有0，1，2代号
class Net2(nn.Module):
    def __init__(self) -> None:
        super(Net2,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3,32,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.dense = nn.Sequential(
            nn.Linear(32*3*3,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
    def forward(self,x):
        conv_out = self.conv(x)
        res = conv_out.view(conv_out.size(0),-1)
        out = self.dense(res)
        return out
net2 = Net2()
print(net2)

Net2(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=288, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [9]:
# 方法三：Sequential + add_module()给容器中的各层赋予对应的名字
class Net3(nn.Module):
    def __init__(self) -> None:
        super(Net3,self).__init__()
        self.conv = nn.Sequential()
        self.conv.add_module("conv1",nn.Conv2d(3,32,3,1,1))
        self.conv.add_module("relu1",nn.ReLU())
        self.conv.add_module("pool1",nn.MaxPool2d(2))
        self.dense = nn.Sequential()
        self.dense.add_module("dense1",nn.Linear(32*3*3,128))
        self.dense.add_module("relu2",nn.ReLU())
        self.dense.add_module("dense2",nn.Linear(128,10))
    def forward(self,x):
        conv_out2 = self.conv(x)
        res = conv_out2.view(conv_out2.size(0),-1)
        out = self.dense(res)
        return out

net3 = Net3()
print(net3)

Net3(
  (conv): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (dense1): Linear(in_features=288, out_features=128, bias=True)
    (relu2): ReLU()
    (dense2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [12]:
# 方法四：Sequential + OrderedDict 时序容器+有序字典的方式，对层结构和各自对应的的名字统一完成构建
class Net4(nn.Module):
    def __init__(self) -> None:
        super(Net4,self).__init__()
        self.block = nn.Sequential(
            OrderedDict(
                [
                    ("conv1",nn.Conv2d(3,32,3,1,1)),
                    ("relu_1",nn.ReLU()),
                    ("pool1",nn.MaxPool2d(2))
                ]
            )
        )
        self.block2 = nn.Sequential(
            OrderedDict(
                [
                    ("dense1",nn.Linear(32*3*3,128)),
                    ("relu_2",nn.ReLU()),
                    ("dense2",nn.Linear(128,10))
                ]
            )
        )
        
    def forward(self,x):
        conv_out2 = self.block(x)
        res = conv_out2.view(conv_out2.size(0),-1)
        out = self.block2(res)
        return out
net4 = Net4()
print(net4)

Net4(
  (block): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (dense1): Linear(in_features=288, out_features=128, bias=True)
    (relu_2): ReLU()
    (dense2): Linear(in_features=128, out_features=10, bias=True)
  )
)
